In [27]:
import ee
import geemap
import spyndex

In [28]:
# ee.Authenticate()
ee.Initialize()

In [29]:
Map = geemap.Map()
roi = geemap.shp_to_ee(r"行政区划.shp").geometry().buffer(3000)
# Map.addLayer(roi, {}, 'roi')
# Map

In [30]:
def maskL457sr(image):
    # Bit 0 - Fill
    # Bit 1 - Dilated Cloud
    # Bit 2 - Unused
    # Bit 3 - Cloud
    # Bit 4 - Cloud Shadow
    qaMask = image.select('QA_PIXEL').bitwiseAnd(int('11111', 2)).eq(0)
    saturationMask = image.select('QA_RADSAT').eq(0)

    # Apply the scaling factors to the appropriate bands.
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBand = image.select('ST_B6').multiply(0.00341802).add(149.0)

    # Replace the original bands with the scaled ones and apply the masks.
    return image.addBands(opticalBands, None, True) \
    .addBands(thermalBand, None, True) \
    .updateMask(qaMask) \
    .updateMask(saturationMask)

def computation(collection):
    N = collection.select(['SR_B4'])
    B = collection.select(['SR_B1'])
    G = collection.select(['SR_B2'])
    R = collection.select(['SR_B3'])
    S1 = collection.select(['SR_B5'])
    S2 = collection.select(['SR_B7'])

    ndvi = spyndex.computeIndex(index=['NDVI'],params={'N': N,'R': R}).rename('ndvi')
    savi = spyndex.computeIndex(index=['SAVI'],params={'L': 0.5,'N': N,'R': R}).rename('savi')
    ndwi = spyndex.computeIndex(index=['NDWI'],params={'G': G,'N': N}).rename('ndwi')
    msavi = spyndex.computeIndex(index=['MSAVI'],params={'N': N,'R': R}).rename('msavi')
    tvi = spyndex.computeIndex(index=['TVI'],params={'N': N,'R': R}).rename('tvi')
    ndpi = spyndex.computeIndex(index=['NDPI'],params={'N': N,'alpha':0.74,'R': R,'S1': S1}).rename('ndpi')
    sr = spyndex.computeIndex(index=['SR'],params={'N': N,'R': R}).rename('sr')
    bndvi = spyndex.computeIndex(index=['BNDVI'],params={'N': N,'B': B}).rename('bndvi')

    band = [ndvi, savi, ndwi, msavi, tvi, ndpi, sr, bndvi, N, B, G, R, S1]
    return ee.Image(band)


In [4]:
img = ee.ImageCollection("LANDSAT/LT05/C02/T1_L2") \
            .filterBounds(roi) \
            .filterDate('1985-01-01', '1988-01-01') \
            .map(maskL457sr) \
            .map(computation) \
            .median() \
            .clip(roi)

In [31]:
img = ee.ImageCollection("LANDSAT/LT05/C02/T1_L2") \
            .filterBounds(roi) \
            .filterDate('2008-01-01', '2009-01-01') \
            .map(maskL457sr) \
            .map(computation) \
            .median() \
            .clip(roi)

In [32]:
img = img.select(['ndvi', 'savi', 'ndwi', 'msavi', 'tvi', 'ndpi', 'sr', 'bndvi', 'SR_B4', 'SR_B1', 'SR_B2', 'SR_B3', 'SR_B5'],
                 ['ndvi', 'savi', 'ndwi', 'msavi', 'tvi', 'ndpi', 'sr', 'bndvi', 'N', 'B', 'G', 'R', 'S1'])

In [24]:
Map.addLayer(img, {}, 'img')
Map

Map(bottom=812.0, center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Search…

In [33]:
for i in [ 'N', 'B', 'G', 'R', 'S1']:
    band = img.select(i)
    geemap.download_ee_image(band, f"{i}.tif", roi, crs='EPSG:4527', scale=30, max_tile_dim=2000)

N.tif: |          | 0.00/35.4M (raw) [  0.0%] in 00:00 (eta:     ?)

B.tif: |          | 0.00/35.4M (raw) [  0.0%] in 00:00 (eta:     ?)

G.tif: |          | 0.00/35.4M (raw) [  0.0%] in 00:00 (eta:     ?)

R.tif: |          | 0.00/35.4M (raw) [  0.0%] in 00:00 (eta:     ?)

S1.tif: |          | 0.00/35.4M (raw) [  0.0%] in 00:00 (eta:     ?)